# Дополнительные инструменты глубокого обучения

## Функциональный API Keras

До сих пор, все рассмотренные в нашем курсе сети реализовывались с использованием последовательной модели Sequential.
Sequential делает предположение о том, что сеть имеет в точности один вход и один выход, и она состоит из линейного стека слоев:
![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/sequential_model.png)

Это очень простое и популярное предположение: эта конфигурация по факту наиболее общая и покрывает большинство практических задач. Но тем не менее, оно слишком негибкое в ряде случаем.

Так, некоторые сети требуют несколько независимых входов, некоторые требуют несколько выходов и некоторые имеют внутреннюю ветвистость между слоями, делая их похожими на графы слоев, а не на линейный стек слоев.

Некоторые задачи, например, трубуют мультимодальные входы: они объедигяют данные, которые пришли из разных источников, обрабатывают каждый тип данных, используя разные види слоёв нейронной сети. Представьте глубоку модель, которая пытается предсказать  аиболее вероятную цену на рынке "секонд-хэнда" используя сл. вход:
* Некоторые метаданные, предоставляемые пользователями (бренд, возраст, и т.д.);
* Текстовое описание, предоставляемое пользователем;
* Изображение товара;

Если у нас есть только метаданные, можно использовать что-то похожее на унитарное кодирование и полносвязную сеть для предсказания цены. Если мы имеем только текстовое описание - используем RNN или 1D convnet. Если только картинку - мы можем использовать 2D convnet.

**Но как заставить обрабатывать все три типа данных одновременно?**

Наивный подход предполагает обучить 3 сепарабельные модели, и затем использовать взвешенное среднее их предсказаний. Однако, это субоптимальное решение, поскольку информация извлекаемая этими моделями является чрезвычайно избыточной.

Более оптимальным способом является **совместное** обучение более точной модели данных с использованием модели, которая может рассматривать входные данные всех модальностей одновременно: модель с тремя входными ветвями:

![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/3-input_model.png)

Похожим образом, некоторые задачи требуют предсказания нескольких целевых атрибутов входных данных. Так, по тексту некоторого хужожественного произведения, например, хотят предсказать не только жанр (роман, триллер и т.д.), но и предсказать примерную дану, когда оно было написано. Разумеется, можно было бы обучить 2 нейронные сети, одну для жанра, а другую для даты. Но поскольку эти атрибты не являются статистически независимыми, можно построить лучшую модель для обучения **совместного предсказания** как жанра, так и даты в одно и тоже время.

Такие совместные модели будут иметь два выходных параметра. Благодаря корреляции между жанром и датой, знание даты поможет модели обучиться более полным и точным репрезентациям пространства жанров и дат.

![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/2-head_model.png) 

Также следует отметить, что много нейросетевых архитектур требует нелинейной сетевой топологии: т.е. сети структуризованные как ориентированные ациклические графы. 

Inception family of networks (Google), базируются на "Inception modules", где вход обрабатывается несколькими параллельными сверточными ветвями, чьи выходы затем объединяются в единый тензор. 

![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/inception_module.png)

Есть также последние тренды в добавлении "residual connections" к модели, которые начинаются с семейства сетей ResNet. 

Residual connections состоят из перепроецирования предыдущих репрезентаций (reinjecting previous representations) в единый поток данных, добавлением предыдущего выходного тензора к более позднему тензору, что помогает предотвратить потерю информации по потоку обработки данных. 
![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/residual_connection.png)

Эти три важных кейса: 
* мультимодальный вход;
* мультимодальный выход;
* графовые представления моделей;

**Невозможны** с использованием только последовательной модели Sequential в Keras. Но есть также другой, более общий и гибкий подход к использованию Keras: функциональный API.

### Введение в функциональный API

В функциональном API происходит прямое манипулирование тензорами, и вы используете слои как функции, который принимают тензоры и возвращают тензоры (отсюда и название функциональный).


In [1]:
from keras import Input, layers

# This is a tensor.
input_tensor = Input(shape=(32,))

# A layer is a function.
dense = layers.Dense(32, activation='relu')

# A layer may be called on a tensor, and it returns a tensor.
output_tensor = dense(input_tensor)

Using TensorFlow backend.


Начнем с простого примера:

Рассмотрим простую Sequential модель и её эквивалент в функциональном API.


In [2]:
from keras.models import Sequential, Model
from keras import layers
from keras import Input

# A Sequential model, which you already know all about.
seq_model = Sequential()
seq_model.add(layers.Dense(32, activation='relu', input_shape=(64,)))
seq_model.add(layers.Dense(32, activation='relu'))
seq_model.add(layers.Dense(10, activation='softmax'))

# Its functional equivalent.
input_tensor = Input(shape=(64,))
x = layers.Dense(32, activation='relu')(input_tensor)
x = layers.Dense(32, activation='relu')(x)
output_tensor = layers.Dense(10, activation='softmax')(x)

# The Model class turns an input tensor and output tensor into a model
model = Model(input_tensor, output_tensor)

# Let's look at it!
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 64)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 32)                2080      
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dense_7 (Dense)              (None, 10)                330       
Total params: 3,466
Trainable params: 3,466
Non-trainable params: 0
_________________________________________________________________


Keras будет извлекать каждый слой, вовлеченный в процесс, начиная с 
    input_tensor
и заканчивая
    output_tensor
соединяя их вместе в подобие графовой структуры - Model.

Разумеется, причина по которой она работает заключается в том, что output_tensor действительно получается путем многократного преобразования input_tensor.

Если вы попытаетесь построить модель из входов и выходов, не связанных друг с другом, вы получите ошибку
    RuntimeError:

In [3]:
unrelated_input = Input(shape=(32,))
bad_model = model = Model(unrelated_input, output_tensor)


RuntimeError: Graph disconnected: cannot obtain value for tensor Tensor("input_2:0", shape=(?, 64), dtype=float32) at layer "input_2". The following previous layers were accessed without issue: []

Эта ошибка в сущности говорит о том, что Keras не способе достичь input_1 из выходного тензора.

При комбиляции, обучении или оценки такой модели Model, API тот же самый, что и при Sequential:


In [4]:
# Compile the model
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Generate dummy Numpy data to train on
import numpy as np
x_train = np.random.random((1000, 64))
y_train = np.random.random((1000, 10))

# Train the model for 10 epochs
model.fit(x_train, y_train, epochs=10, batch_size=128)

# Evaluate the model
score = model.evaluate(x_train, y_train)

Epoch 1/10
1000/1000 [==============================] - 0s - loss: 11.6939     
Epoch 2/10
1000/1000 [==============================] - 0s - loss: 11.6167     
Epoch 3/10
1000/1000 [==============================] - 0s - loss: 11.6061     
Epoch 4/10
1000/1000 [==============================] - 0s - loss: 11.5999     
Epoch 5/10
1000/1000 [==============================] - 0s - loss: 11.5959     
Epoch 6/10
1000/1000 [==============================] - 0s - loss: 11.5922     
Epoch 7/10
1000/1000 [==============================] - 0s - loss: 11.5897     
Epoch 8/10
1000/1000 [==============================] - 0s - loss: 11.5871     
Epoch 9/10
1000/1000 [==============================] - 0s - loss: 11.5848     
Epoch 10/10
  32/1000 [..............................] - ETA: 0s

## Модель с несколькими входами

Функциональный API может быть использован для построения моделей, использующих несколько входов.

Обычно такие модели в некоторой части будут "объединять" входные ветви с использованием специального слоя, который комбинирует несколько тензоров, т.е. добавляет их, проводит конкатенацию и т.д.

Это обычно выполняетя посредством операций объединения (merge operation) таких как:
* keras.layers.add
* keras.layers.concatenate
* etc.

Рассмотрим простую модель с несколькими входами: модель вопросно-ответной системы.

Обычная вопросно-ответная модель имеет 2 входа: вопрос на естественном языке и текстовый сниппет (например, новостная статья), предоставляющая информацию, которая должна быть использована при ответе на вопрос. 

Модель должна произвести ответ: в самом простейшем случае это просто однословный ответ полученный через применение softmax над некоторым предобределенным словарем.

![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/qa_model.png)

Пример, как мы можем построить подобную модель с функциональным API:
Мы устанавливаем 2 независимых ветви, кодируя текстовый вход и вопрос как вектора репрезентации, затем мы объединяем эти вектора и добавляет softmax классификатор:

In [5]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

# Our text input is a variable-length sequence of integers.
# Note that we can optionally name our inputs!
text_input = Input(shape=(None,), dtype='int32', name='text')

# Which we embed into a sequence of vectors of size 64
embedded_text = layers.Embedding(64, text_vocabulary_size)(text_input)

# Which we encoded in a single vector via a LSTM
encoded_text = layers.LSTM(32)(embedded_text)

# Same process (with different layer instances) for the question
question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(32, question_vocabulary_size)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

# We then concatenate the encoded question and encoded text
concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

# And we add a softmax classifier on top
answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

# At model instantiation, we specify the two inputs and the output:
model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])

Как мы обучим такую модель?

Есть два возможных API: вы можете передать как вход в модель список NumPy массивов, или вы можете ередать словарь, отображающий входные имена на NumPy массивам.

Разумеется, последняя опция доступна лишь в том случае, если вы даете имена входам:

In [ ]:
import numpy as np

num_samples = 1000
max_length = 20

# Let's generate some dummy Numpy data
text = np.random.randint(1, text_vocabulary_size, size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size, size=(num_samples, max_length))

# Answers are one-hot encoded, not integers
answers = np.random.randint(0, 1, size=(num_samples, answer_vocabulary_size))

# Fitting using a list of inputs
model.fit([text, question], answers, epochs=10, batch_size=128)

# Fitting using a dictionary of inputs (only if inputs were named!)
model.fit({'text': text, 'question': question}, answers,
          epochs=10, batch_size=128)

## Модель с несколькими выходами

Таким же образом, функциональный API может быть использован для построения моделей с несколькими выходами. Простой пример, который строит сеть, которая пытается одновременно предсказать разные свойства данных: например, сеть которая берет на вход серию постов из социальных медиа от одной анонимной персоны и пытается предсказать атрибуты:
* возраст;
* пол;
* уровень дохода;
![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/social_media_3-head_model.png)

In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 50000
num_income_groups = 10

posts_input = Input(shape=(None,), dtype='int32', name='posts')
embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu')(embedded_posts)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.MaxPooling1D(5)(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.Conv1D(256, 5, activation='relu')(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dense(128, activation='relu')(x)

# Note that we are giving names to the output layers.
age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups, activation='softmax', name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)

model = Model(input_posts, [age_prediction, income_prediction, gender_prediction])

Обучение такой моедли требует возможности указать **разные функции потерь** для разных выходов сети:
* возраст рассматривается как задача регрессии
* пол как задача бинарной классификации, которая требует отдельной процедуры обучеия.

Однако, градиентный спуск требует от нас минимизировать скаляр, мы должны скомбинировать эти функции потерь в единой значение чтобы обучать модель. Простейший способ комбинирования разных функций потерь состоит в том, чтобы просто суммировать их. В Keras вы можете либо задать список, или словарь функций потерь, который передается в compile для указания разных объектов для разных выходных значений и результирующие значения функции потерь суммируются в глобальные потери, которые минимизируются в процессе обучения.

In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'])

# Equivalent (only possible if you gave names to the output layers!):
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'})

Заметим, что также возможно назначить разную важность для функций потерь в их вкладе в финальну. потерю. Это полезно в частности если разные потери принимают значения на разных шкалах. Например, MSE потеря используемая для задачи возрастной регрессии может иметь типичное значение погрежности 305, в то время, как потрея перекрестной энтропии для задачи классификации гендера может достигать 0.1 

В такой ситуации чтобы определить вклад разных функций потерь для более тщательного балансирования, мы назначим вес 10 для кросс-энтропии и вес 0.25 для MSE.

In [ ]:
model.compile(optimizer='rmsprop',
              loss=['mse', 'categorical_crossentropy', 'binary_crossentropy'],
              loss_weights=[0.25, 1., 10.])

# Equivalent (only possible if you gave names to the output layers!):
model.compile(optimizer='rmsprop',
              loss={'age': 'mse',
                    'income': 'categorical_crossentropy',
                    'gender': 'binary_crossentropy'},
              loss_weights={'age': 0.25,
                            'income': 1.,
                            'gender': 10.})

Как и в случае моделей с несколькими входами, передача данных NumPy в модель для обучения может осуществляться либо через список массивов, либо через словарь массивов:


In [ ]:
# age_targets, income_targets and gender_targets are assumed to be Numpy arrays
model.fit(posts, [age_targets, income_targets, gender_targets],
          epochs=10, batch_size=64)

# Equivalent (only possible if you gave names to the output layers!):
model.fit(posts, {'age': age_targets,
                  'income': income_targets,
                  'gender': gender_targets},
          epochs=10, batch_size=64)

## Ориентированные ацикличные графы слоёв

С фунциональным API можно реализовывать сети с очень сложной внутренней топологией. Нейронные сети в Keras позволяют определять произвольные **ориентированные ациклические графы** слоёв.

Ацикличность важна - эти графы не могут содержать циклов! Возможны только петли (в рекуррентных связях) которые являются внутренней структурой рекуррентных слоев.

Несколько общих компонент нейронных сетей реализованы как графы.
* Inception modules;
* residual connections;

Для лучщего понимания как функциональный API строит графы слоев рассмотрим реализацию обоих архитектур в Keras.

### Inception modules

Inception - популярная архитектура для сверточных нейронных сетей, разработанная Christian Szegedy в компании Google в 2013-2014 года, вдохновленная более ранней архитектурай "network in network". Она состоит из стека моулей, который сами по себе выглядят как малые независимые сети, разделенные на несколько параллельных ветвей. Наиболее базовая форма inception модуля имеет от 3 до 4 ветвей, начинающихся с конволюции 1x1, последующей 3x3 и заканчивающейся конкатенацией результирующих признаков. Эта настройка помогает сети независимо обучаться пространственным атрибутам и channel-wise атрибутам, которые более эффективны чем их совместное обучение. Более сложные версии Inception модуля также возможны, и как правило включают в себя операции пулинга, различные пространственные размеры конволюции например (5x5 вместо 3x3 для некоторых ветвей) и ветви без пространственной конволюции (1x1 конволюция). Пример:
![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/inception_module.png)
Реализация Inception модуля с функциональным API:


In [ ]:
from keras import layers

# We assume the existence of a 4D input tensor `x`

# Every branch has the same stride value (2), which is necessary to keep all
# branch outputs the same size, so as to be able to concatenate them.
branch_a = layers.Conv2D(128, 1, activation='relu', strides=2)(x)

# In this branch, the striding occurs in the spatial convolution layer
branch_b = layers.Conv2D(128, 1, activation='relu')(x)
branch_b = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_b)

# In this branch, the striding occurs in the average pooling layer
branch_c = layers.AveragePooling2D(3, strides=2, activation='relu')(x)
branch_c = layers.Conv2D(128, 3, activation='relu')(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu')(x)
branch_d = layers.Conv2D(128, 3, activation='relu')(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', strides=2)(branch_d)

# Finally, we concatenate the branch outputs to obtain the module output
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)

Полная архитектура Inception V3 доступна в Keras как
    keras.applications.inception_v3.InceptionV3
включает веса предобученной модели на наборе данных ImageNet.

Еще одной тесносвязанной моделью, доступной как часть Keras является модуль Xception (переводится как extreme inception), и являющийся архитектурой вдохновленной Inception. Он берет за основу идею разделения обучения по каналам (channel-wise) и по пространственным атрибутам (space-wise) и возводит её до логической экстремы - заменяя Incption модули углубленными разделимыми свертками (depthwise separable convolutions), состоящие из углубленных конволюций (пространственная конволюция, где каждый канал обрабатывается отдельно), за которыми следуют 1x1 конволюции - эффективная, экстремальная форма Inception модуля, где пространственные атрибуты и канальные атрибуты полностью разделены.

Xception имеет примерно одинаковое число параметров как и Inception V3, но показывает лучшую производительность по времени выполнения и более высокую точность на ImageNet, благодаря более эффективному использованию параметров

#### Residual connections

Residual connections - очень общая графоподобная сетевая компонента, впервые описанная примерно в 2015 году. Эти архитектурные компоненты были введены He et al (Microsoft). Они обрабатывали 2 общие проблемы, возникающие в любой масштабируемой модели глубокого обучения: затухающий градиент и representation bottlenecks.

Добавление residual connections **к любой модели имеющей более 10 слоев**, в общем случае, приводит к положительным результатам.

Residual connection просто заключается в создание выхода на раннем слое, доступном как вход более позднего слоя, эффективно создавая кратчайший путь в последовательной сети. Вместо того, чтобы быть объединенным в более позднюю активацю, более ранний выход суммируется с более поздней активацией, которая предполагает что обе активации имеют равный размер. В случае разных размеров можно использовать линейное преобразование для перемасштабирования ранней активации в целевой размер.


Реализация residual connection когда размеры карты атрибутов совпадают: использование идентичных residual connections

In [ ]:
from keras import layers

# We assume the existence of a 4D input tensor `x`
x = ...
# We apply some transformation to `x`
y = layers.Conv2D(128, 3, activation='relu')(x)
y = layers.Conv2D(128, 3, activation='relu')(y)
y = layers.Conv2D(128, 3, activation='relu')(y)

# We add the original `x` back to the output features
y = layers.add([y, x])

Реализация residual connections, когда размеры карты атрибутов отличаются: использование линейного преобращования:

In [ ]:
from keras import layers

# We assume the existence of a 4D input tensor `x`
x = ...
y = layers.Conv2D(128, 3, activation='relu')(x)
y = layers.Conv2D(128, 3, activation='relu')(y)
y = layers.MaxPooling2D(2, strides=2)(y)

# We use a 1x1 convolution to linearly downsample
# the original `x` tensor to the same shape as `y`
residual = layers.Conv2D(1, strides=2)(x)

# We add the residual tensor back to the output features
y = layers.add([y, residual])

## Разделение весов в слоях

Одной важной особенностью функционального API является возможность повторного использования экземпляра слоя несколько раз. Когда вы вызываете экземпляр слоя дважды, экземпляр создания нового слоя для каждого вызова вы повторно используете веса при каждом вызове. (When you call a layer instance twice, instance of instantiating a new layer for each call, you are reusing the same weights with every call.)

Это позволяет построить модели имеющие разделяемые ветви - несколько ветвей, которые разделяют общие знания и выполняют те же операции, т.е. разделяют одни и те же репрезентации и используют эти репрезентации одновременно для разных множеств входных значений.

В качестве примера можно привести модель, которая пытается оценить семантическую схожесть между двумя предложениями. Модель имеет 2 входа (2 предложения) и порождает вещественное число между 0 и 1 (0 - несвязанные предложения, 1 - предложения или идентичны, или чистая переформулировка друг друга). Такая модель может быть полезной во многих преложениях.

В этом случае, 2 входных предложения взаимозаменяемы, поскольку семантическая схоесть является симметричным отношением - схожесть A и B идентична схожести B и A. По этим же причинам, нет смысла обучать 2 независимые модели для обработки каждого входного предложения. Вместо этого мы бы хотели обработать с помощью единой LSTM сети. Репрезентация LSTM слоя (его весов) будет обучаться на базе обоих входов одновременно. Это то что называется "разделяемой LSTM".

Здесь мы будем реализовывать такую модель с использованием разделения весов слоев в функциональном API Keras.


In [ ]:
from keras import layers
from keras import Input
from keras.models import Model

# We instantiate a single LSTM layer, once
lstm = layers.LSTM(32)

# Building the left branch of the model
# -------------------------------------

# Inputs are variable-length sequences of vectors of size 128
left_input = Input(shape=(None, 128))
left_output = lstm(left_input)

# Building the right branch of the model
# --------------------------------------

right_input = Input(shape=(None, 128))
# When we call an existing layer instance,
# we are reusing its weights
right_output = lstm(right_input)

# Building the classifier on top
# ------------------------------

merged = layers.concatenate([left_output, right_output], axis=-1)
predictions = layers.Dense(1, activation='sigmoid')(merged)

# Instantiating and training the model
# ------------------------------------

model = Model([left_input, right_input], predictions)
# When you train such a model, the weights of the `lstm` layer
# are updated based on both inputs.
model.fit([left_data, right_data], targets)

Естественно, экземпляр слоя может использоваться более одного раза - его можно вызвать произвольное количество раз, каждый раз повторно используя одно и то же множество весов.


### Модели как слои

Важно то, что в функциональном API модели могут использоватькак если бы вы использовали слои - т.е. можно думать о модели как о "большом слое". Это верно как для класса Sequential так и о Model. Это означает что вы можете вызвать модель на входном тензоне и передать её выходному тензору.

Пример: модели как слои, т.е. модели как функции:

In [ ]:
y = model(x)

Если модель имеет несколько входных тензоров и несколько выъходных тензоров, она должна быть вызвана со списком тензоров:

In [ ]:
y1, y2 = model([x1, x2])

При вызове экземпляра модели, вы повторно используете веса модели - в точности как когда вы используете экземпляр слоя. Простым вызовом экземпляра, не важно, является ли экземпляр слоем или модель, будет всегда повторно использоваться уже существующая обученнаая репрезентация объекта - что достаточно интуитивно.

Одним простым практическим примером является то, что Вы можете построить повторно используемый экземпляр модели - компьютерное зрение, которое использует на входе двойную камеру - две параллельные камеры, стоящие в отдалии несколько сантиметров друг от друга. 

По факту вы не нуждаетесь в двух параллельных моделях, для экстракции визуальных атрибутов от левой камеры и от правой камеры перед объедиением этих двух потоков. Такая низкоуровневая обработка может быть разделена среди двумя входами, т.е. выполняться слоями, которые используют одни и те же веса, и которые могут разделять одни и те же репрезентации.

Пример в Keras:

In [ ]:
from keras import layers
from keras import applications
from keras import Input

# Our base image processing model with be the Xception network
# (convolutional base only).
xception_base = applications.Xception(weights=None, include_top=False)

# Our inputs are 250x250 RGB images.
left_input = Input(shape=(250, 250, 3))
right_input = Input(shape=(250, 250, 3))

# We call the same vision model twice!
left_features = xception_base(left_input)
right_input = xception_base(right_input)

# The merged features contain information from both
# the right visual feed and the left visual feed
merged_features = layers.concatenate([left_features, right_input], axis=-1)

# Максимальная отдача от ваших моделей

Попытка построения архитектур вслепую может "хорошо" работать в одном случае:
![](http://memesmix.net/media/created/mfmd05.jpg)

Но иногда требуется более эффективная модель, чем "и так сойдет" и которая будет давать великолепные результаты и даже побеждать в соревнованиях.

Дадим краткий обзор техник для построения современных моделей глубокого обучения.

## Продвинутые архитектурные паттерны

Мы уже познакомились с одним важным архитектурным паттерном прежде: residual connections. Рассмотрим еще два достаточно важных архитектурных паттерна.

Эти паттерны особенно релевантны при построении высокопроизводительных глубоких сверточных сетей, но также могут быть найдены во многих других типах архитектур.

### Batch Normalization

Нормализация это широкая категория методов, которые направлены на то, чтобы сделать разные объекты, передаваемые модели машинного обучения более похожими друг на друга, что поможет модели обучиться и обобщить выводы на новые данные. Наиболее общей формой нормализаци иданных является центрорование данных около нулевого значения посредством вычитания среднего и преобразование к стандартному отклонению посредством деление данных на их стандартное отклонение. 

Фактически, делается предположение что данные сооветствуют нормальному распределению и обеспечивается чтобы это распределение было центрировано и масштабировано по стандартному отклонению:

In [ ]:
normalized_data = (data - np.mean(data, axis=...)) / np.std(data, axis=...)

В предыдущих примерах мы проводили нормализацию данных перед передачей их в модели. Однако, нормализаиция данных должна по прежнему происходить после каждой операции преобразования в сети: даже если данные, идущие от Dense или Conv2D имеют нормализацию по стандартному отклонению, нет никаких оснований ожидать априори что они будут оставаться в той же форме после прохождения через жти слои.

Batch normalization - это слой типа (BatchNormalization в Keras) введенный в 2015 году (Ioffe, Szegedy), способный адаптивно нормализовать данные, даже когда их среднее значение и дисперсия изменяются в процессе обучения. 
It works by internally maintaining an exponential moving average of the batch-wise mean and variance of the data seen during training. 

Основной эффект batch normalization состоит в том, что она помогаетраспространению градиента (gradient propagation) - подобно residual connections - и поэтому применима для глубоких сетей. Некоторые глубокие сети могут быть обучены только если они включают несколько слоев BatchNormalization.

Например, BatchNormalization достаточно свободно используется в большинстве современных конволюционных архитектур, таких как ResNet50, InceptionV3 и Xception.

Слой BatchNormalization обычно используется после конволюционного или полносвязного слоя:

In [ ]:
# After a Conv layer:
conv_model.add(layers.Conv2D(32, 3, activation='relu'))
conv_model.add(layers.BatchNormalization())

# After a Dense layer:
dense_model.add(layers.Dense(32, activation='relu'))
dense_model.add(layers.BatchNormalization())

Слой BatchNormalization принимает на вход аргумент axis, который указывает оси атрибутов, которые должны быть нормализованы. По умолчанию это число равно -1, последняя оси во входном тензоре.

Это корректное значение при использовании слоев Dense, слоев Conv1D, Conv2D, RNN слоев с data_format установленным на channels_last.

Однако, в случае использования Conv2D с data_format установленым в значение channels_first, оси атрибутов имеют axis number 1, и axis аргумент в BatchNormalization должен быть соответственно установлен в 1.

Недавним усовершенствованием обычно batch normalization была т.н. batch renormalization (2017). Он предлагает преимущества по сравнению с batch normalization, без каких либо очевидных затрат. Пока еще слишком рано говорить о преимуществах, и будет ли она вытеснять привычную регуляризацию.

Совсем недавно Klambauer и др. ввели т.н. "самонастраивающиеся нейронные сети" (self-normaliung neural networks), которым удалось управлять нсохранением нормализации данных, после прохождения через Dense слой, посредством специфической функции активации (selu) и специфического инциализатора (lecun_normal. Эта схема выглядит интересной, но сейчас она ограничена полносвязными нейронными сетями.

### Depthwise Separable Convolution

Есть ли слой, который вы можете использовать в качестве замены для Conv2D который делает модель более легкой (меньшее число весовых параметров), быстрее (меньшее число операций с плавающей точкой)?

Именно это делает слой: **depthwise separable convolution layer (SeparableConv2D)**.  

Слой depthwise separable convolution выполняет пространственную свертку (spatial convolution) на каждом канале входа, независимо(!), перед смешиванием выходных каналов через "поточечную" свертку (1x1 convolution). Это эквивалентно разделению обучения пространственных атрибутов и обучению chanel-wise атрибут, что имеет смысл если допустить, что пространственные локации входа сильно коррелированы, в то время как разные каналы могут быть независимы. Слой требует намного меньше параметров, и включает в себя меньше вычислений, и результатом является меньшая и более быстрая модель.

И поскольку это более репрезентативный и эффективный способ выполнения свертки, он, как правило, учит лучшим репрезентациям, используя меньше данных, что приводит к более эффективным моделям.

![](https://dpzbhybb2pdcj.cloudfront.net/chollet/v-6/Figures/depthwise_separable_conv.png)

Эти преимущества становятся особенно эффективными, когда мы обучаем небольшие модели с нуля на ограниченных данных.

Например, построим легковесную depthwise separable convnet для задачи классификации изображений (softmax categorial classification) на небольшом наборе данных:


In [ ]:
model = Sequential()
model.add(SeparableConv2D(32, activation='relu', input_shape=(height, width, channels)))
model.add(SeparableConv2D(64, activation='relu'))
model.add(MaxPooling2D(2))

model.add(SeparableConv2D(64, activation='relu'))
model.add(SeparableConv2D(128, activation='relu'))
model.add(MaxPooling2D(2))

model.add(SeparableConv2D(64, activation='relu'))
model.add(SeparableConv2D(128, activation='relu'))
model.add(GlobalAveragePooling2D())

model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

When it comes to larger-scale models, depthwise separable convolutions are the basis of the Xception architecture, a high-performing convnet that comes packaged with Keras. 

### Оптимизация гиперпараметров
При построении моделей глубокого обучения, есть много решений которые нужно принять:
* как много нужно выбрать слоев?
* как много нейронов или фильтров должно быть на каждом слое?
* какую функцию активации выбрать?
* Нужно ли использовать BatchNormalization после уровня?
* Какой dropout использовать?
* ...

Эти архитектурные параметры называются "гиперпараметрами", которые отличаются от параметров модели, которые настраиваются посредством обратного распространения ошибки.

Формальных правил к оптимизации параметров не существует. Ваши первоначальные решения почти всегда субоптимальны, даже если у вас хорошая интуиция. Вы можете уточнить их, изменив их параметры вручную и повторно обучив свою модель - на самом деле это то, что большую часть времени проводят инженеры по машинному обучению и исследователи.

Необходимо исследовательно пространство всевозможных решений автоматически и системно. Необходимо проводить поиск в пространстве архитектурных парамтров и находить наиболее лучшую модель эмпирически. Область автоматической оптимизации гиперпараметров - достаточно важная область исследований.

Процесс оптимизации гиперпараметров обычно выглядит следующим образом:
* Выберете набор гиперпараметров (автоматически);
* Постройте соответствующую модель;
* Обучите на тренировочных данных и измерьте производительность на валидационных данных;
* Выберете следующее множество гиперпараметров для тестирования (автоматически);
* Повторите;
* ...
* В конце концов измерьте производительность на тестовых данных.

Ключевым моментов в этом процессе является алгоритм, который использует историю оценок производительности по серии гиперпараметров для выбора следующео набора гиперпараметров для оценки. Возможно множество техник: байесовская оптимизация, генетические алгоритмы и рандомизированный поиск...

Обучение весов модели относительно простая задача - просто вычисляется функция потерь на mini-batches данных, затем используется алгоритм обратного распространения ошибки для "продвижения" весов в правильном направлении. Обновление гиперпараметров, с другой стороны, является достаточно тяжелой задачей.

* Вычисление сигнала обратной связи (множество гиперпараметров приводит ли к более высокой производительности модели для данной задачи?) и может быть исключтельно дорогостоящим с точки зрения вычислительной мозности: он требует создания и обучения новой модели каждый раз с нуля;
* Пространство гиперпараметров обычно делается как дискретные решения, и не является непрерывным и дифференцируемым. Пожтому невозможно применить градиентный спуск в пространстве гиперпараметров. Вместо этого нужно ориентироваться на техники оптимизации, не задействующие градиент, которые являются менее эффективными чем градиентный спуск.

Из-за этих особенностей в настоящее время мы очень ограничены в инструментарии для оптимизации моделей. Часто выбор происходит посредством случайного поиска. ОДнако один инструмент, который можно применить - это *Hyperopt* - библиотека на Python для оптимизации гиперпараметров, которая внутри использует деревья для подбора. Еще одна библиотека называется Hyperas, которая интегрирует Hyperopt для использования с моделями Keras.

Одной важной проблемой, которую надо держать в голове при автоматическом подборе параметров - переобучение на валидационной выборке. По мере обновления гиперпараметров на базе сигналов, вычисляемых при использовании валидационных данных, вы можете обучиться на валидационных данных и поэтому быстро переобучиться на них. Следует держать это в голове.

Итак, оптимизация гиперпараметров эффективная техника, которая является необходимым требованием для получения современных моедлей практически для любой задачи или соревнования.


### Ансамбли моделей